# analyze video(s)
Various cells for analyzing videos, groups of videos, etc...

In [1]:
# inits
import subprocess
import glob
import yaml
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))  # add partent directory
%load_ext autoreload
%autoreload 2

root_dir = r'D:\github\fish-tracking'  # github repo

# format command for analyze_video.py
def make_cmd(video, model_name, skeleton,
             batch_size=16, make_tracking_vid=False):
    cmd = 'python analyze_video.py {} {} {} -batch_size {}'.format(
        video, model_name, skeleton, batch_size)
    if overwrite:
        cmd += ' -overwrite'
    if make_tracking_vid:
        cmd += ' -makevideo'
    return cmd

## analyze single video
analyze a single video, and make video with tracking overlaid. must specify the model and skeleton.

In [14]:
# analyze single video

# settings
video = r'Z:\C1\chin_for_rick\201119_C1stimwNS\201119_8.avi'
# model_name = r'D:\github\fish-tracking\train\models\tank_model.h5'
# skeleton = r'D:\github\fish-tracking\label\skeletons\tank_skeleton_3.csv'
model_name = r'D:\github\fish-tracking\train\models\stim_model.h5'
skeleton = r'D:\github\fish-tracking\label\skeletons\stim_skeleton.csv'
overwrite = True
make_tracking_vid = True
batch_size = 16

cmd = make_cmd(video, model_name, skeleton, batch_size, make_tracking_vid)
msg = subprocess.run(cmd)

## analyze all folders for project
given project name (`tank` or `stim`) analyzes all videos. assumes the following folder structure for the projections:
- todo
- todo

edit `config.yaml` to define folders, models, skeletons...

`todo`
- make sure folders hierarchies are correct for different projects
- mechanism for skipping old folders...

In [ ]:
# settings
vid_type = 'stim'   # 'tank' or 'stim'
overwrite = False   # whether to overwrite existing tracking
batch_size = 16
make_tracking_vid = True

# load config
with open(os.path.join(root_dir, 'config.yaml'), 'r') as file:
    cfg = yaml.safe_load(file)[vid_type]

# find video folders
vid_folders = [f.path for f in os.scandir(cfg['data_dir']) if f.is_dir()]

# analyze videos
for vid_folder in vid_folders:
    
    # don't analyze vids that are concatenated, or tracking videos
    vids = glob.glob(os.path.join(vid_folder, '*.avi'))
    vids = [vid for vid in vids if 'concatenated' not in vid and 'tracking' not in vid]
    vids.sort()

    for idx, vid in enumerate(vids):
        print('\n({}/{})--------- analyzing {}'.format(idx, len(vids), vid))
        cmd = make_cmd(vid, cfg['model'], cfg['skeleton'], 
                      batch_size, make_tracking_vid)
        subprocess.run(cmd)





(0/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_10.avi

(1/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_100.avi

(2/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_102.avi

(3/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_103.avi

(4/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_105.avi

(5/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_106.avi

(6/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_108.avi

(7/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_109.avi

(8/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_11.avi

(9/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_111.avi

(10/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_112.avi

(11/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_114.avi

(12


(101/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_216.avi

(102/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_217.avi

(103/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_219.avi

(104/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_22.avi

(105/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_220.avi

(106/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_222.avi

(107/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_223.avi

(108/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_224.avi

(109/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_225.avi

(110/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_226.avi

(111/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_227.avi

(112/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wN


(199/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_309.avi

(200/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_310.avi

(201/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_311.avi

(202/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_312.avi

(203/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_313.avi

(204/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_314.avi

(205/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_315.avi

(206/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_316.avi

(207/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_317.avi

(208/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_318.avi

(209/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_319.avi

(210/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_w


(296/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_40.avi

(297/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_400.avi

(298/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_401.avi

(299/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_402.avi

(300/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_403.avi

(301/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_404.avi

(302/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_405.avi

(303/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_406.avi

(304/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_407.avi

(305/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_408.avi

(306/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_409.avi

(307/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wN


(394/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_76.avi

(395/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_78.avi

(396/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_79.avi

(397/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_8.avi

(398/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_81.avi

(399/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_82.avi

(400/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_84.avi

(401/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_85.avi

(402/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_87.avi

(403/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_88.avi

(404/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_90.avi

(405/411)--------- analyzing Z:\C1\chin_for_rick\191206_C1stim_wNS\20191206_

...


(183/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_71.avi

(184/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_72.avi

(185/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_73.avi

(186/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_74.avi

(187/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_75.avi

(188/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_76.avi

(189/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_77.avi

(190/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_78.avi

(191/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_79.avi

(192/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_8.avi

(193/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_80.avi

(194/214)--------- analyzing Z:\C1\chin_for_rick\200302_c1stimwNS\20200303_81.avi

(195


(64/70)--------- analyzing Z:\C1\chin_for_rick\200517_distal_tracing_stim\20200517_7.avi

(65/70)--------- analyzing Z:\C1\chin_for_rick\200517_distal_tracing_stim\20200517_70.avi

(66/70)--------- analyzing Z:\C1\chin_for_rick\200517_distal_tracing_stim\20200517_71.avi

(67/70)--------- analyzing Z:\C1\chin_for_rick\200517_distal_tracing_stim\20200517_72.avi

(68/70)--------- analyzing Z:\C1\chin_for_rick\200517_distal_tracing_stim\20200517_8.avi

(69/70)--------- analyzing Z:\C1\chin_for_rick\200517_distal_tracing_stim\20200517_9.avi

(0/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_10.avi

(1/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_100.avi

(2/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_101.avi

(3/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_102.avi

(4/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_103.avi

(5/256)--------- analyzing Z:\C1\chin_for_


(95/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_186.avi

(96/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_187.avi

(97/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_188.avi

(98/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_189.avi

(99/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_19.avi

(100/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_190.avi

(101/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_191.avi

(102/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_192.avi

(103/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_193.avi

(104/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_194.avi

(105/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_195.avi

(106/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\202


(193/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_40.avi

(194/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_41.avi

(195/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_42.avi

(196/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_43.avi

(197/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_44.avi

(198/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_45.avi

(199/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_46.avi

(200/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_47.avi

(201/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_48.avi

(202/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_49.avi

(203/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527_50.avi

(204/256)--------- analyzing Z:\C1\chin_for_rick\200527_c1stim_wNS\20200527


(15/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_113.avi

(16/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_114.avi

(17/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_115.avi

(18/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_116.avi

(19/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_117.avi

(20/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_12.avi

(21/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_13.avi

(22/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_14.avi

(23/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_15.avi

(24/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_16.avi

(25/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_17.avi

(26/109)--------- analyzing Z:\C1\chin_for_rick\200617_stim_wNS\200617_18.avi

(27/109)--------- analyzing Z:\C1\chin_for_ric